En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

Este archivo fue creado en google Colab, si desea ejecutarlo en otro entorno puede ser necesario instalar las siguientes dependencias:

In [ ]:
#!pip install gdown pandas pyspark

In [1]:
import gdown
import pandas as pd
import zipfile
import pyspark

from pandas.io import gbq
from pandas import json_normalize

Obtengo el archivo desde google drive y lo descomprimo

In [2]:
file_path = "farmers-protest-tweets-2021-2-4.json"
url = f'https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis'
gdown.download(url, 'tweets.zip', quiet=False)

extracted_dir = 'unzip/'

# Create a ZipFile object and extract the contents
with zipfile.ZipFile('tweets.zip', 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)


Downloading...
From (uriginal): https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis
From (redirected): https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis&confirm=t&uuid=9ca5fde4-cc01-4704-bca3-8f9be1ecd733
To: c:\Users\Caruso\source\Latam-DE-challenge\src\tweets.zip
100%|██████████| 60.4M/60.4M [00:03<00:00, 17.3MB/s]


Inicializo el cluster de spark
https://www.youtube.com/watch?v=_C8kWso4ne4

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FarmersProtest").getOrCreate()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [13]:
df_tweets = pd.read_json(extracted_dir+file_path, lines=True)

# Tengo que normalizar el objeto user para poder acceder a sus atributos
df_tweets['userName'] = json_normalize(df_tweets['user']).username

df_tweets['date'] = pd.to_datetime(df_tweets['date']).dt.date
# Agrupo por 'userName' y 'date' y cuento las filas para saber cuantos tweets hizo cada usuario por día.
df_tweets = df_tweets.groupby(['userName','date']).size().reset_index(name='countByUserByDay')
# Ahora calculo la cantidad de tweets por día sin agrupar por usuario.
df_tweets['countByDay'] = df_tweets.groupby('date')['countByUserByDay'].transform('sum')
# Transformo la columna 'date' a tipo datetime


# Con la función Rank marco el usuario con más tweets cada día, desempato usando first.
df_tweets['dailyUserRank'] = df_tweets.groupby('date')['countByUserByDay'].rank(ascending=False, method='first')

df_tweets[(df_tweets['dailyUserRank'] == 1)].sort_values(by='countByDay', ascending=False).head(10)[['date','userName','countByDay']]

,userName,date,countByDay
23626,RanbirS00614606,2021-02-12,12347
16761,MaanDee08215437,2021-02-13,11296
22650,RaaJVinderkaur,2021-02-17,11087
40318,jot__b,2021-02-16,10443
46423,rebelpacifist,2021-02-14,10249
44085,neetuanjle_nitu,2021-02-18,9625
40317,jot__b,2021-02-15,9197
17343,MangalJ23056160,2021-02-20,8502
28820,Surrypuria,2021-02-23,8417
21929,Preetm91,2021-02-19,8204
